# Demonstrate how to replace population and mortality data for Alzheimer's Model 3 with forecasts

    Age-specific population - /mnt/share/forecasting/data/9/future/population/20240320_daly_capstone_resubmission_squeeze_soft_round_shifted_hiv_shocks_covid_all_who_reagg/population_agg.nc

    Deaths Rates - /snfs1/Project/forecasting/results/7/future/death/20240320_daly_capstone_resubmission_squeeze_soft_round_shifted_hiv_shocks_covid_all_who_reagg/



In [32]:
import vivarium
!date

Thu Aug 14 10:36:28 PDT 2025


In [33]:
import xarray as xr, numpy as np, pandas as pd, matplotlib.pyplot as plt

In [34]:
# copy model 2 artifact to a new place for modification
# TODO: make this work for all location artifacts
old_artifact_path = '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model2.0/china.hdf'
new_artifact_dir = '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model3.0tyler/'
!cp $old_artifact_path $new_artifact_dir   # TODO: modify an artifact being built for the current model instead of copying the artifact from model 2
!ls -halt $new_artifact_dir/china.hdf

-rw-r--r-- 1 tylerdy IHME-Simulationscience 2.9M Aug 14 10:36 /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model3.0tyler//china.hdf


In [35]:
# take a look at what we are planning to replace
art = vivarium.Artifact(
    f"{new_artifact_dir}/china.hdf"
)  # TODO: make this generate for any location
art.keys

['metadata.keyspace',
 'metadata.locations',
 'population.location',
 'population.structure',
 'population.age_bins',
 'population.demographic_dimensions',
 'population.theoretical_minimum_risk_life_expectancy',
 'cause.all_causes.cause_specific_mortality_rate',
 'covariate.live_births_by_sex.estimate',
 'cause.alzheimers_disease_and_other_dementias.prevalence_scale_factor',
 'cause.alzheimers_disease_and_other_dementias.prevalence',
 'cause.alzheimers_disease_and_other_dementias.incidence_rate',
 'cause.alzheimers_disease_and_other_dementias.cause_specific_mortality_rate',
 'cause.alzheimers_disease_and_other_dementias.excess_mortality_rate',
 'cause.alzheimers_disease_and_other_dementias.disability_weight',
 'cause.alzheimers_disease_and_other_dementias.restrictions']

In [36]:
# art.load('population.structure')

In [37]:
# art.load('cause.all_causes.cause_specific_mortality_rate')

## make tables from FHS .nc files

In [38]:
def table_from_nc(fname_dict, param, loc_id, loc_name, age_mapping):
    ds = xr.open_dataset(
        fname_dict[param],
        engine="netcdf4",  # let xarray auto-detect; list here if you know it
        decode_cf=True,  # handle CF-conventions (time units, etc.)
    )

    # Select relevant part of FHS dataset
    var_name = param
    if param == "births":
        var_name = "population"
    elif param in ["migration", "mortality"]:
        var_name = "value"

    pop_ts = ds[var_name].sel(
        location_id=loc_id,
    )

    if param != "migration":
        pop_ts = pop_ts.isel(scenario=0)

    pop_ts = pop_ts.squeeze(drop=True)  # remove now-singleton dims

    df = pop_ts.to_dataframe(name="value").reset_index()

    # Transform to vivarium format
    # 1. Convert location_id to location name
    df["location"] = loc_name

    # 2. Convert sex_id to sex names
    sex_mapping = {
        1: "Male",
        2: "Female",
    }
    df["sex"] = df["sex_id"].map(sex_mapping)

    # 3. Convert age_group_id to age intervals
    if param != "births":
        age_bins = age_mapping.set_index("age_group_id")
        df["age_start"] = np.round(df["age_group_id"].map(age_bins["age_start"]), 3)
        df["age_end"] = np.round(df["age_group_id"].map(age_bins["age_end"]), 3)
        age_cols = ["age_start", "age_end"]
    else:
        age_cols = []

    # 4. Convert year_id to year intervals
    df["year_start"] = df["year_id"].map(int)
    df["year_end"] = df["year_id"].map(int) + 1

    # 5. Set index and unstack to get draw columns
    index_cols = (
        [
            "location",
            "sex",
        ]
        + age_cols
        + ["year_start", "year_end", "draw"]
    )
    df_indexed = df.dropna(subset=index_cols).set_index(index_cols)

    df_wide = df_indexed["value"].unstack(level="draw")

    # 6. Rename columns to draw_x format
    df_wide.columns = [f"draw_{col}" for col in df_wide.columns]

    return df_wide

In [39]:
fname_dict = {
    "population": "/mnt/share/forecasting/data/9/future/population/20240320_daly_capstone_resubmission_squeeze_soft_round_shifted_hiv_shocks_covid_all_who_reagg/population_agg.nc",
    #     'births': '/mnt/share/forecasting/data/9/future/live_births/20231204_ref/live_births.nc',
    #     'deaths': '/snfs1/Project/forecasting/results/7/future/death/20240320_daly_capstone_resubmission_squeeze_soft_round_shifted_hiv_shocks_covid_all_who_reagg_num/_all.nc',
    "mortality": "/snfs1/Project/forecasting/results/7/future/death/20240320_daly_capstone_resubmission_squeeze_soft_round_shifted_hiv_shocks_covid_all_who_reagg/_all.nc",
    #     'migration': '/mnt/share/forecasting/data/6/future/migration/20230605_loc_intercept_shocks_pg_21LOCS_ATTENUATED/migration.nc',
}

# These IDs are listed on https://shiny.ihme.washington.edu/content/273/
loc_id, loc_name = (
    6,
    "China",
)  # TODO: make this work for any location
age_mapping = pd.read_csv(
    f"data/age_bins.csv"
)  # downloaded with a different environment
# using vivarium_inputs.utility_data.get_age_bins;
# see also https://shiny.ihme.washington.edu/content/273/
# TODO: refactor this so that it does not require an extra .csv file

In [40]:
%%time

df = {}

for param in ['population', 'mortality']:
    df[param] = table_from_nc(fname_dict, param, loc_id, loc_name, age_mapping)  # slow, takes minutes to hours to run, depending on disk caching

CPU times: user 6.48 s, sys: 2.64 s, total: 9.12 s
Wall time: 1min 31s


In [41]:
art.load("population.structure")  # old population structure

value
location sex    age_start age_end    year_start year_end              
China    Female 0.000000  0.019178   2021       2022      9.700551e+04
                0.019178  0.076712   2021       2022      2.917973e+05
                0.076712  0.500000   2021       2022      2.206675e+06
                0.500000  1.000000   2021       2022      2.713134e+06
                1.000000  2.000000   2021       2022      6.130422e+06
                2.000000  5.000000   2021       2022      2.459860e+07
                5.000000  10.000000  2021       2022      4.482746e+07
                10.000000 15.000000  2021       2022      4.020045e+07
                15.000000 20.000000  2021       2022      3.458934e+07
                20.000000 25.000000  2021       2022      3.430665e+07
                25.000000 30.000000  2021       2022      4.086633e+07
                30.000000 35.000000  2021       2022      5.846949e+07
                35.000000 40.000000  2021       2022      5.159715e+07
                40.000000 45.000000  2021       2022      4.460838e+07
                45.000000 50.000000  2021       2022      5.426008e+07
                50.000000 55.000000  2021       2022      5.971800e+07
                55.000000 60.000000  2021       2022      5.505881e+07
                60.000000 65.000000  2021       2022      3.637982e+07
                65.000000 70.000000  2021       2022      3.896343e+07
                70.000000 75.000000  2021       2022      2.743995e+07
                75.000000 80.000000  2021       2022      1.751651e+07
                80.000000 85.000000  2021       2022      1.111688e+07
                85.000000 90.000000  2021       2022      6.044836e+06
                90.000000 95.000000  2021       2022      2.123153e+06
                95.000000 125.000000 2021       2022      5.181548e+05
         Male   0.000000  0.019178   2021       2022      1.130040e+05
                0.019178  0.076712   2021       2022      3.398118e+05
                0.076712  0.500000   2021       2022      2.569386e+06
                0.500000  1.000000   2021       2022      3.158248e+06
                1.000000  2.000000   2021       2022      7.114943e+06
                2.000000  5.000000   2021       2022      2.833526e+07
                5.000000  10.000000  2021       2022      5.094293e+07
                10.000000 15.000000  2021       2022      4.599251e+07
                15.000000 20.000000  2021       2022      4.008281e+07
                20.000000 25.000000  2021       2022      3.886750e+07
                25.000000 30.000000  2021       2022      4.561574e+07
                30.000000 35.000000  2021       2022      6.268379e+07
                35.000000 40.000000  2021       2022      5.436611e+07
                40.000000 45.000000  2021       2022      4.692531e+07
                45.000000 50.000000  2021       2022      5.606148e+07
                50.000000 55.000000  2021       2022      6.114104e+07
                55.000000 60.000000  2021       2022      5.488356e+07
                60.000000 65.000000  2021       2022      3.662535e+07
                65.000000 70.000000  2021       2022      3.774008e+07
                70.000000 75.000000  2021       2022      2.585649e+07
                75.000000 80.000000  2021       2022      1.560243e+07
                80.000000 85.000000  2021       2022      8.674975e+06
                85.000000 90.000000  2021       2022      3.480915e+06
                90.000000 95.000000  2021       2022      8.088347e+05
                95.000000 125.000000 2021       2022      1.209437e+05

In [42]:
df["population"]["value"] = df["population"].mean(axis=1)
new_pop_structure = df["population"].filter(like="value")

In [43]:
new_pop_structure

value
location sex    age_start age_end year_start year_end              
China    Female 0.0       5.0     2021       2022      3.071707e+07
                                  2022       2023      2.770277e+07
                                  2023       2024      2.441780e+07
                                  2024       2025      2.153625e+07
                                  2025       2026      1.945937e+07
...                                                             ...
         Male   95.0      125.0   2046       2047      4.555385e+05
                                  2047       2048      5.045809e+05
                                  2048       2049      5.643234e+05
                                  2049       2050      6.236108e+05
                                  2050       2051      6.736043e+05

[1200 rows x 1 columns]

In [44]:
art.replace("population.structure", new_pop_structure)
art.load("population.structure")  # new population structure

value
location sex    age_start age_end year_start year_end              
China    Female 0.0       5.0     2021       2022      3.071707e+07
                                  2022       2023      2.770277e+07
                                  2023       2024      2.441780e+07
                                  2024       2025      2.153625e+07
                                  2025       2026      1.945937e+07
...                                                             ...
         Male   95.0      125.0   2046       2047      4.555385e+05
                                  2047       2048      5.045809e+05
                                  2048       2049      5.643234e+05
                                  2049       2050      6.236108e+05
                                  2050       2051      6.736043e+05

[1200 rows x 1 columns]

In [45]:
art.load("cause.all_causes.cause_specific_mortality_rate")  # old mortality rates

draw_0    draw_1    draw_2  \
sex    age_start age_end    year_start year_end                                 
Female 0.000000  0.019178   2021       2022      0.133283  0.106464  0.101824   
       0.019178  0.076712   2021       2022      0.015074  0.012081  0.011868   
       0.076712  0.500000   2021       2022      0.004426  0.003890  0.003714   
       0.500000  1.000000   2021       2022      0.001495  0.001025  0.000842   
       1.000000  2.000000   2021       2022      0.001266  0.000894  0.000727   
       2.000000  5.000000   2021       2022      0.000195  0.000139  0.000107   
       5.000000  10.000000  2021       2022      0.000194  0.000167  0.000150   
       10.000000 15.000000  2021       2022      0.000147  0.000137  0.000142   
       15.000000 20.000000  2021       2022      0.000218  0.000208  0.000224   
       20.000000 25.000000  2021       2022      0.000291  0.000292  0.000352   
       25.000000 30.000000  2021       2022      0.000307  0.000307  0.000379   
       30.000000 35.000000  2021       2022      0.000426  0.000427  0.000524   
       35.000000 40.000000  2021       2022      0.000636  0.000636  0.000777   
       40.000000 45.000000  2021       2022      0.001020  0.001021  0.001244   
       45.000000 50.000000  2021       2022      0.001423  0.001421  0.001733   
       50.000000 55.000000  2021       2022      0.002461  0.002455  0.002990   
       55.000000 60.000000  2021       2022      0.003899  0.003890  0.004720   
       60.000000 65.000000  2021       2022      0.006640  0.006614  0.007882   
       65.000000 70.000000  2021       2022      0.011386  0.011356  0.013351   
       70.000000 75.000000  2021       2022      0.021327  0.021291  0.024785   
       75.000000 80.000000  2021       2022      0.037255  0.037211  0.043041   
       80.000000 85.000000  2021       2022      0.072692  0.072710  0.082061   
       85.000000 90.000000  2021       2022      0.121898  0.121960  0.138918   
       90.000000 95.000000  2021       2022      0.205045  0.205096  0.227701   
       95.000000 125.000000 2021       2022      0.312495  0.311717  0.352537   
Male   0.000000  0.019178   2021       2022      0.165524  0.132622  0.128411   
       0.019178  0.076712   2021       2022      0.017523  0.014031  0.013924   
       0.076712  0.500000   2021       2022      0.005541  0.004819  0.004746   
       0.500000  1.000000   2021       2022      0.002298  0.001607  0.001351   
       1.000000  2.000000   2021       2022      0.001484  0.001059  0.000886   
       2.000000  5.000000   2021       2022      0.000257  0.000171  0.000140   
       5.000000  10.000000  2021       2022      0.000328  0.000274  0.000249   
       10.000000 15.000000  2021       2022      0.000243  0.000238  0.000246   
       15.000000 20.000000  2021       2022      0.000381  0.000414  0.000468   
       20.000000 25.000000  2021       2022      0.000635  0.000713  0.000829   
       25.000000 30.000000  2021       2022      0.000776  0.000847  0.000950   
       30.000000 35.000000  2021       2022      0.001075  0.001177  0.001329   
       35.000000 40.000000  2021       2022      0.001477  0.001647  0.001895   
       40.000000 45.000000  2021       2022      0.002137  0.002427  0.002852   
       45.000000 50.000000  2021       2022      0.002675  0.003092  0.003711   
       50.000000 55.000000  2021       2022      0.004178  0.004856  0.005863   
       55.000000 60.000000  2021       2022      0.006192  0.007221  0.008739   
       60.000000 65.000000  2021       2022      0.010063  0.011616  0.013882   
       65.000000 70.000000  2021       2022      0.016031  0.018491  0.022007   
       70.000000 75.000000  2021       2022      0.029528  0.033714  0.039676   
       75.000000 80.000000  2021       2022      0.050529  0.057149  0.066214   
       80.000000 85.000000  2021       2022      0.095433  0.106342  0.119987   
       85.000000 90.000000  2021       2022      0.226706  0.244970  0.265172   
 

In [46]:
df["mortality"]

draw_0    draw_1  \
location sex    age_start age_end year_start year_end                       
China    Female 0.0       5.0     2022       2023      0.000289  0.000294   
                                  2023       2024      0.000275  0.000278   
                                  2024       2025      0.000263  0.000266   
                                  2025       2026      0.000253  0.000255   
                                  2026       2027      0.000242  0.000244   
...                                                         ...       ...   
         Male   95.0      125.0   2046       2047      0.338828  0.331181   
                                  2047       2048      0.339337  0.329314   
                                  2048       2049      0.340084  0.328945   
                                  2049       2050      0.342007  0.327594   
                                  2050       2051      0.343720  0.327547   

                                                         draw_2    draw_3  \
location sex    age_start age_end year_start year_end                       
China    Female 0.0       5.0     2022       2023      0.000322  0.000293   
                                  2023       2024      0.000307  0.000277   
                                  2024       2025      0.000290  0.000266   
                                  2025       2026      0.000277  0.000257   
                                  2026       2027      0.000266  0.000248   
...                                                         ...       ...   
         Male   95.0      125.0   2046       2047      0.305841  0.298271   
                                  2047       2048      0.305519  0.298321   
                                  2048       2049      0.303552  0.297854   
                                  2049       2050      0.302546  0.296576   
                                  2050       2051      0.301797  0.295353   

                                                         draw_4    draw_5  \
location sex    age_start age_end year_start year_end                       
China    Female 0.0       5.0     2022       2023      0.000309  0.000301   
                                  2023       2024      0.000294  0.000286   
                                  2024       2025      0.000280  0.000267   
                                  2025       2026      0.000271  0.000253   
                                  2026       2027      0.000263  0.000243   
...                                                         ...       ...   
         Male   95.0      125.0   2046       2047      0.378265  0.311104   
                                  2047       2048      0.382098  0.310468   
                                  2048       2049      0.382799  0.306865   
                                  2049       2050      0.381641  0.303975   
                                  2050       2051      0.381678  0.302285   

                                                         draw_6    draw_7  \
location sex    age_start age_end year_start year_end                       
China    Female 0.0       5.0     2022       2023      0.000298  0.000459   
                                  2023       2024      0.000283  0.000411   
                                  2024       2025      0.000269  0.000384   
                                  2025       2026      0.000255  0.000367   
                                  2026       2027      0.000247  0.000349   
...                                                         ...       ...   
         Male   95.0      125.0   2046       2047      0.309795  0.294326   
                                  2047       2048      0.312317  0.292889   
                                  2048       2049      0.313474  0.290964   
                                  2049       2050      0.312148  0.287102   
                                  2050       2051      0.313261  0.285237   

                                                 

In [47]:
art.replace(
    "cause.all_causes.cause_specific_mortality_rate",
    df["mortality"].loc["China"],
)
art.load("cause.all_causes.cause_specific_mortality_rate")  # new mortality rates

draw_0    draw_1    draw_2  \
sex    age_start age_end year_start year_end                                 
Female 0.0       5.0     2022       2023      0.000289  0.000294  0.000322   
                         2023       2024      0.000275  0.000278  0.000307   
                         2024       2025      0.000263  0.000266  0.000290   
                         2025       2026      0.000253  0.000255  0.000277   
                         2026       2027      0.000242  0.000244  0.000266   
...                                                ...       ...       ...   
Male   95.0      125.0   2046       2047      0.338828  0.331181  0.305841   
                         2047       2048      0.339337  0.329314  0.305519   
                         2048       2049      0.340084  0.328945  0.303552   
                         2049       2050      0.342007  0.327594  0.302546   
                         2050       2051      0.343720  0.327547  0.301797   

                                                draw_3    draw_4    draw_5  \
sex    age_start age_end year_start year_end                                 
Female 0.0       5.0     2022       2023      0.000293  0.000309  0.000301   
                         2023       2024      0.000277  0.000294  0.000286   
                         2024       2025      0.000266  0.000280  0.000267   
                         2025       2026      0.000257  0.000271  0.000253   
                         2026       2027      0.000248  0.000263  0.000243   
...                                                ...       ...       ...   
Male   95.0      125.0   2046       2047      0.298271  0.378265  0.311104   
                         2047       2048      0.298321  0.382098  0.310468   
                         2048       2049      0.297854  0.382799  0.306865   
                         2049       2050      0.296576  0.381641  0.303975   
                         2050       2051      0.295353  0.381678  0.302285   

                                                draw_6    draw_7    draw_8  \
sex    age_start age_end year_start year_end                                 
Female 0.0       5.0     2022       2023      0.000298  0.000459  0.000212   
                         2023       2024      0.000283  0.000411  0.000230   
                         2024       2025      0.000269  0.000384  0.000185   
                         2025       2026      0.000255  0.000367  0.000175   
                         2026       2027      0.000247  0.000349  0.000164   
...                                                ...       ...       ...   
Male   95.0      125.0   2046       2047      0.309795  0.294326  0.308151   
                         2047       2048      0.312317  0.292889  0.306731   
                         2048       2049      0.313474  0.290964  0.305385   
                         2049       2050      0.312148  0.287102  0.303144   
                         2050       2051      0.313261  0.285237  0.301571   

                                                draw_9  ...  draw_490  \
sex    age_start age_end year_start year_end            ...             
Female 0.0       5.0     2022       2023      0.000281  ...  0.000278   
                         2023       2024      0.000264  ...  0.000264   
                         2024       2025      0.000252  ...  0.000250   
                         2025       2026      0.000239  ...  0.000238   
                         2026       2027      0.000228  ...  0.000228   
...                                                ...  ...       ...   
Male   95.0      125.0   2046       2047      0.325364  ...  0.340285   
                         2047       2048      0.325635  ...  0.338628   
                         2048       2049      0.325331  ...  0.337658   
                         2049       2050      0.323233  ...  0.336868   
                         2050       2051      0.322160  ...  0.336583   

                                              draw_49

In [48]:
new_pop_bins = art.load("population.age_bins").query("age_start >= 5.0")
art.replace("population.age_bins", new_pop_bins)

In [49]:
# now we need to replicate the prevalence scale factor for each year
df = art.load("cause.alzheimers_disease_and_other_dementias.prevalence_scale_factor")
df

draw_0    draw_1    draw_2  \
sex    age_start age_end    year_start year_end                                 
Female 0.000000  0.019178   2021       2022      0.000000  0.000000  0.000000   
       0.019178  0.076712   2021       2022      0.000000  0.000000  0.000000   
       0.076712  0.500000   2021       2022      0.000000  0.000000  0.000000   
       0.500000  1.000000   2021       2022      0.000000  0.000000  0.000000   
       1.000000  2.000000   2021       2022      0.000000  0.000000  0.000000   
       2.000000  5.000000   2021       2022      0.000000  0.000000  0.000000   
       5.000000  10.000000  2021       2022      0.000000  0.000000  0.000000   
       10.000000 15.000000  2021       2022      0.000000  0.000000  0.000000   
       15.000000 20.000000  2021       2022      0.000000  0.000000  0.000000   
       20.000000 25.000000  2021       2022      0.000000  0.000000  0.000000   
       25.000000 30.000000  2021       2022      0.000000  0.000000  0.000000   
       30.000000 35.000000  2021       2022      0.000000  0.000000  0.000000   
       35.000000 40.000000  2021       2022      0.000000  0.000000  0.000000   
       40.000000 45.000000  2021       2022      0.000230  0.000252  0.000146   
       45.000000 50.000000  2021       2022      0.001722  0.001561  0.001064   
       50.000000 55.000000  2021       2022      0.004591  0.003711  0.003426   
       55.000000 60.000000  2021       2022      0.008160  0.007749  0.007667   
       60.000000 65.000000  2021       2022      0.015748  0.016415  0.013433   
       65.000000 70.000000  2021       2022      0.025865  0.032316  0.027762   
       70.000000 75.000000  2021       2022      0.052195  0.058204  0.051549   
       75.000000 80.000000  2021       2022      0.125949  0.124671  0.099461   
       80.000000 85.000000  2021       2022      0.252330  0.238615  0.211083   
       85.000000 90.000000  2021       2022      0.338760  0.304324  0.313941   
       90.000000 95.000000  2021       2022      0.316515  0.405180  0.387618   
       95.000000 125.000000 2021       2022      0.331304  0.474157  0.472265   
Male   0.000000  0.019178   2021       2022      0.000000  0.000000  0.000000   
       0.019178  0.076712   2021       2022      0.000000  0.000000  0.000000   
       0.076712  0.500000   2021       2022      0.000000  0.000000  0.000000   
       0.500000  1.000000   2021       2022      0.000000  0.000000  0.000000   
       1.000000  2.000000   2021       2022      0.000000  0.000000  0.000000   
       2.000000  5.000000   2021       2022      0.000000  0.000000  0.000000   
       5.000000  10.000000  2021       2022      0.000000  0.000000  0.000000   
       10.000000 15.000000  2021       2022      0.000000  0.000000  0.000000   
       15.000000 20.000000  2021       2022      0.000000  0.000000  0.000000   
       20.000000 25.000000  2021       2022      0.000000  0.000000  0.000000   
       25.000000 30.000000  2021       2022      0.000000  0.000000  0.000000   
       30.000000 35.000000  2021       2022      0.000000  0.000000  0.000000   
       35.000000 40.000000  2021       2022      0.000000  0.000000  0.000000   
       40.000000 45.000000  2021       2022      0.000187  0.000231  0.000123   
       45.000000 50.000000  2021       2022      0.001462  0.001420  0.000904   
       50.000000 55.000000  2021       2022      0.003997  0.003269  0.002936   
       55.000000 60.000000  2021       2022      0.007097  0.006838  0.006674   
       60.000000 65.000000  2021       2022      0.013612  0.014550  0.011391   
       65.000000 70.000000  2021       2022      0.020355  0.026084  0.021222   
       70.000000 75.000000  2021       2022      0.034671  0.041175  0.035101   
       75.000000 80.000000  2021       2022      0.079729  0.085402  0.064701   
       80.000000 85.000000  2021       2022      0.169845  0.163067  0.140367   
       85.000000 90.000000  2021       2022      0.242441  0.215879  0.223926   
 

In [50]:
index_cols = list(df.index.names)
df = df.reset_index()

In [51]:
df_list = []

for y in range(2022, 2051):
    df_y = df.copy()
    df_y["year_start"] = y
    df_y["year_end"] = y + 1
    df_list.append(df_y)

In [52]:
df_all = pd.concat(df_list).set_index(index_cols)
df_all

draw_0    draw_1    draw_2  \
sex    age_start age_end    year_start year_end                                 
Female 0.000000  0.019178   2022       2023      0.000000  0.000000  0.000000   
       0.019178  0.076712   2022       2023      0.000000  0.000000  0.000000   
       0.076712  0.500000   2022       2023      0.000000  0.000000  0.000000   
       0.500000  1.000000   2022       2023      0.000000  0.000000  0.000000   
       1.000000  2.000000   2022       2023      0.000000  0.000000  0.000000   
...                                                   ...       ...       ...   
Male   75.000000 80.000000  2050       2051      0.079729  0.085402  0.064701   
       80.000000 85.000000  2050       2051      0.169845  0.163067  0.140367   
       85.000000 90.000000  2050       2051      0.242441  0.215879  0.223926   
       90.000000 95.000000  2050       2051      0.224280  0.295808  0.281282   
       95.000000 125.000000 2050       2051      0.222614  0.337319  0.332564   

                                                   draw_3    draw_4    draw_5  \
sex    age_start age_end    year_start year_end                                 
Female 0.000000  0.019178   2022       2023      0.000000  0.000000  0.000000   
       0.019178  0.076712   2022       2023      0.000000  0.000000  0.000000   
       0.076712  0.500000   2022       2023      0.000000  0.000000  0.000000   
       0.500000  1.000000   2022       2023      0.000000  0.000000  0.000000   
       1.000000  2.000000   2022       2023      0.000000  0.000000  0.000000   
...                                                   ...       ...       ...   
Male   75.000000 80.000000  2050       2051      0.074635  0.076486  0.072128   
       80.000000 85.000000  2050       2051      0.151198  0.137804  0.121565   
       85.000000 90.000000  2050       2051      0.240541  0.185718  0.233090   
       90.000000 95.000000  2050       2051      0.288031  0.303301  0.307365   
       95.000000 125.000000 2050       2051      0.283842  0.360733  0.314798   

                                                   draw_6    draw_7    draw_8  \
sex    age_start age_end    year_start year_end                                 
Female 0.000000  0.019178   2022       2023      0.000000  0.000000  0.000000   
       0.019178  0.076712   2022       2023      0.000000  0.000000  0.000000   
       0.076712  0.500000   2022       2023      0.000000  0.000000  0.000000   
       0.500000  1.000000   2022       2023      0.000000  0.000000  0.000000   
       1.000000  2.000000   2022       2023      0.000000  0.000000  0.000000   
...                                                   ...       ...       ...   
Male   75.000000 80.000000  2050       2051      0.084939  0.073453  0.073782   
       80.000000 85.000000  2050       2051      0.170296  0.149567  0.127238   
       85.000000 90.000000  2050       2051      0.208650  0.219380  0.207965   
       90.000000 95.000000  2050       2051      0.266006  0.225376  0.248562   
       95.000000 125.000000 2050       2051      0.339101  0.245042  0.298472   

                                                   draw_9  ...  draw_490  \
sex    age_start age_end    year_start year_end            ...             
Female 0.000000  0.019178   2022       2023      0.000000  ...  0.000000   
       0.019178  0.076712   2022       2023      0.000000  ...  0.000000   
       0.076712  0.500000   2022       2023      0.000000  ...  0.000000   
       0.500000  1.000000   2022       2023      0.000000  ...  0.000000   
       1.000000  2.000000   2022       2023      0.000000  ...  0.000000   
...                                                   ...  ...       ...   
Male   75.000000 80.000000  2050       2051      0.067720  ...  0.058886   
       80.000000 85.000000  2050       2051      0.119304  ...  0.099902   
       85.000000 90.000000  2050       2051      0.175135  ...  0.172991   
       90.000000 95.000000  2050       2051      0.23

In [28]:
art.replace(
    "cause.alzheimers_disease_and_other_dementias.prevalence_scale_factor", df_all
)

In [29]:
!ls -halt $new_artifact_dir/china.hdf

-rw-r--r-- 1 tylerdy IHME-Simulationscience 9.8M Aug 14 10:07 /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model3.0tyler//china.hdf
